## Agent Chat Lab -  Multi Agent Orchestration

In [ ]:
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.agents import AgentGroupChat
import sys
import os
from dotenv import load_dotenv
sys.path.insert(1,'..') 
from src.agent import Agent

load_dotenv(override=True)

In [ ]:
# // Create agents
AGENT_SCORE = "Score"
AGENT_DEBT = "Dividas"
AGENT_CC_SANTANDER = "CC_Santander"
AGENT_CC_WILLIAMS = "CC_Will"
AGENT_CC_PICPAY = "CC_PicPay"
AGENT_CC_NEON = "CC_Neon"
AGENT_CC_ITAU = "CC_Itau"
AGENT_CC_DM = "CC_DM"
AGENT_CC_DIGIO = "CC_Digio"
AGENT_BING = "Bing"

# // Azure AI Foundry Agent Creator
agent_creator = Agent()

model_deployment_name = os.environ.get("AZURE_OPENAI_4o_MODEL_NAME")

async with():
    agent_1 = await agent_creator.create_agent(
        model=model_deployment_name,
        name=AGENT_SCORE,
        instructions="You answer questions about Score",
    )

    agent_2 = await agent_creator.create_agent(
        model=model_deployment_name,
        name=AGENT_DEBT,
        instructions="You answer questions about Debt",
    )




In [ ]:
# // Selection function
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.
    
    Choose only from these participants:
    - {AGENT_SCORE}
    - {AGENT_DEBT}
    - {AGENT_CC_SANTANDER}
    - {AGENT_CC_WILLIAMS}
    - {AGENT_CC_PICPAY}
    - {AGENT_CC_NEON}
    - {AGENT_CC_ITAU}
    - {AGENT_CC_DM}
    - {AGENT_CC_DIGIO}
    - {AGENT_BING}

    History:
    {{{{$history}}}}
    """,
)